In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 16.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=4d209b2920ec5a1780959799f6412888a26f3ee5ff5b6ee3fa6114827819896c
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark


In [3]:
import pyspark

In [4]:
conf = pyspark.SparkConf().setMaster("local").setAppName("ProjectEDA")
sc = pyspark.SparkContext(conf=conf)

In [5]:
spark = pyspark.sql.SparkSession(sc)

In [6]:
df_temp_data = spark.read.csv('city_temperature.csv', header=True, inferSchema=True)
df_temp_data.take(5)

[Row(Region='Africa', Country='Algeria', State=None, City='Algiers', Month=1, Day=1, Year=1995, AvgTemperature=64.2),
 Row(Region='Africa', Country='Algeria', State=None, City='Algiers', Month=1, Day=2, Year=1995, AvgTemperature=49.4),
 Row(Region='Africa', Country='Algeria', State=None, City='Algiers', Month=1, Day=3, Year=1995, AvgTemperature=48.8),
 Row(Region='Africa', Country='Algeria', State=None, City='Algiers', Month=1, Day=4, Year=1995, AvgTemperature=46.4),
 Row(Region='Africa', Country='Algeria', State=None, City='Algiers', Month=1, Day=5, Year=1995, AvgTemperature=47.9)]

In [7]:
#This prints out the schema of the Dataframe. 
df_temp_data.printSchema()

root
 |-- Region: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- State: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Day: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- AvgTemperature: double (nullable = true)



In [8]:
#This prints out the first 5 rows of the Dataframe. 
df_temp_data.show(5)

+------+-------+-----+-------+-----+---+----+--------------+
|Region|Country|State|   City|Month|Day|Year|AvgTemperature|
+------+-------+-----+-------+-----+---+----+--------------+
|Africa|Algeria| null|Algiers|    1|  1|1995|          64.2|
|Africa|Algeria| null|Algiers|    1|  2|1995|          49.4|
|Africa|Algeria| null|Algiers|    1|  3|1995|          48.8|
|Africa|Algeria| null|Algiers|    1|  4|1995|          46.4|
|Africa|Algeria| null|Algiers|    1|  5|1995|          47.9|
+------+-------+-----+-------+-----+---+----+--------------+
only showing top 5 rows



In [9]:
#This prints out the first 5 rows of the DataFrame, but in row format. 
df_temp_data.head(5)

[Row(Region='Africa', Country='Algeria', State=None, City='Algiers', Month=1, Day=1, Year=1995, AvgTemperature=64.2),
 Row(Region='Africa', Country='Algeria', State=None, City='Algiers', Month=1, Day=2, Year=1995, AvgTemperature=49.4),
 Row(Region='Africa', Country='Algeria', State=None, City='Algiers', Month=1, Day=3, Year=1995, AvgTemperature=48.8),
 Row(Region='Africa', Country='Algeria', State=None, City='Algiers', Month=1, Day=4, Year=1995, AvgTemperature=46.4),
 Row(Region='Africa', Country='Algeria', State=None, City='Algiers', Month=1, Day=5, Year=1995, AvgTemperature=47.9)]

In [10]:
#This counts the number of rows in the DataFrame. 
df_temp_data.count()

2906327

In [11]:
# If you want to view statistical properties of all the columns. However, in this context, it only makes sense for AvgTemperature column
# for col in df_temp_data.columns:
#     df_temp_data.describe([col]).show()

In [12]:
#This determines the statistics of the Average Temperature Column. 
df_temp_data.describe(["AvgTemperature"]).show()

+-------+-----------------+
|summary|   AvgTemperature|
+-------+-----------------+
|  count|          2906327|
|   mean|56.00492078145553|
| stddev|32.12359394724281|
|    min|            -99.0|
|    max|            110.0|
+-------+-----------------+



In [13]:
#This finds unique values of a categorical column. We used Region here as the sample column. 
df_temp_data.select('Region').distinct().rdd.map(lambda r: r[0]).collect()

['Europe',
 'Africa',
 'North America',
 'Middle East',
 'South/Central America & Carribean',
 'Asia',
 'Australia/South Pacific']

In [14]:
#This find the number of unqiue countries.
distinct_countries = df_temp_data.select('Country').distinct().rdd.map(lambda r: r[0]).collect()
len(distinct_countries)

125

In [15]:
#This find the number of unique cities. 
distinct_cities = df_temp_data.select('City').distinct().rdd.map(lambda r: r[0]).collect()
len(distinct_cities)

321

In [16]:
# Practice filtering data based on a column value
us_cities = df_temp_data.filter("Country == 'US'").select('City').distinct().rdd.map(lambda r: r[0]).collect()

# Merging Lat/Long Data for Cities

In this next section, I'm experimenting with pulling latitude/longitude data for each city based on worldcities.csv file found here: https://www.kaggle.com/datasets/juanmah/world-cities

In [17]:
df_city_data = spark.read.csv('worldcities_modified.csv', header=True, inferSchema=True)
df_city_data.show(5)

+---------+----------+--------+--------+-----------+----+----+----------+-------+----------+----------+
|     city|city_ascii|     lat|     lng|    country|iso2|iso3|admin_name|capital|population|        id|
+---------+----------+--------+--------+-----------+----+----+----------+-------+----------+----------+
|    Tokyo|     Tokyo| 35.6839|139.7744|      Japan|  JP| JPN|     Tōkyō|primary|  39105000|1392685764|
|  Jakarta|   Jakarta| -6.2146|106.8451|  Indonesia|  ID| IDN|   Jakarta|primary|  35362000|1360771077|
|    Delhi|     Delhi| 28.6667| 77.2167|      India|  IN| IND|     Delhi|  admin|  31870000|1356872604|
|   Manila|    Manila|    14.6|120.9833|Philippines|  PH| PHL|    Manila|primary|  23971000|1608618140|
|São Paulo| Sao Paulo|-23.5504|-46.6339|     Brazil|  BR| BRA| São Paulo|  admin|  22495000|1076532519|
+---------+----------+--------+--------+-----------+----+----+----------+-------+----------+----------+
only showing top 5 rows



In [18]:
df_city_data.printSchema()

root
 |-- city: string (nullable = true)
 |-- city_ascii: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lng: double (nullable = true)
 |-- country: string (nullable = true)
 |-- iso2: string (nullable = true)
 |-- iso3: string (nullable = true)
 |-- admin_name: string (nullable = true)
 |-- capital: string (nullable = true)
 |-- population: integer (nullable = true)
 |-- id: integer (nullable = true)



In [19]:
df_city_data.count()

12986

In [20]:
# Combine US cities with states because some cities share names
from pyspark.sql.functions import when, concat
df_temp_data = df_temp_data.withColumn("City", when(df_temp_data.Country == "US", concat(df_temp_data["City"], df_temp_data["State"]))
                                 .otherwise(df_temp_data.City))

In [21]:
# Drop State column as it's no longer necessary
df_temp_data = df_temp_data.drop("State")

In [22]:
# Combine US cities with states to match temperature data City column
df_city_data = df_city_data.withColumn("city_ascii", when(df_city_data.country == "US", concat(df_city_data["city_ascii"], df_city_data["admin_name"]))
                                      .otherwise(df_city_data.city_ascii))

In [23]:
# Create a key column to join against worldcities file
df_temp_data = df_temp_data.withColumn("CityKey", concat(df_temp_data["City"], df_temp_data["Country"]))
df_city_data = df_city_data.withColumn("Key", concat(df_city_data["city_ascii"], df_city_data["country"]))

In [24]:
df_temp_data.count()

2906327

In [25]:
# Join the temp dataframe with the cities dataframe
combined_data = df_temp_data.join(df_city_data, df_temp_data.CityKey == df_city_data.Key, "left").select(df_temp_data["*"], df_city_data.lat, df_city_data.lng)
combined_data.show(5)

+------+-------+-------+-----+---+----+--------------+--------------+-------+------+
|Region|Country|   City|Month|Day|Year|AvgTemperature|       CityKey|    lat|   lng|
+------+-------+-------+-----+---+----+--------------+--------------+-------+------+
|Africa|Algeria|Algiers|    1|  1|1995|          64.2|AlgiersAlgeria|36.7764|3.0586|
|Africa|Algeria|Algiers|    1|  2|1995|          49.4|AlgiersAlgeria|36.7764|3.0586|
|Africa|Algeria|Algiers|    1|  3|1995|          48.8|AlgiersAlgeria|36.7764|3.0586|
|Africa|Algeria|Algiers|    1|  4|1995|          46.4|AlgiersAlgeria|36.7764|3.0586|
|Africa|Algeria|Algiers|    1|  5|1995|          47.9|AlgiersAlgeria|36.7764|3.0586|
+------+-------+-------+-----+---+----+--------------+--------------+-------+------+
only showing top 5 rows



In [26]:
# Verify that no city is missing lat/long data
test = combined_data.filter("lat IS NULL").select('CityKey').distinct().rdd.map(lambda r: r[0]).collect()
test

[]

In [27]:
if df_temp_data.count() != combined_data.count():
    print("Error: Number of rows changed after join!")
else:
    print("Join operation successful!")

Join operation successful!


In [28]:
# Check for any cities that may have received two sets of coordinates or more. Should be empty if done correctly.
from pyspark.sql.functions import countDistinct, col
combined_data.groupBy("CityKey").agg(countDistinct('lat').alias('GeoCount')).filter(col('GeoCount') > 1).show(truncate=False)

+-------+--------+
|CityKey|GeoCount|
+-------+--------+
+-------+--------+



In [29]:
combined_data.show(5)

+------+-------+-------+-----+---+----+--------------+--------------+-------+------+
|Region|Country|   City|Month|Day|Year|AvgTemperature|       CityKey|    lat|   lng|
+------+-------+-------+-----+---+----+--------------+--------------+-------+------+
|Africa|Algeria|Algiers|    1|  1|1995|          64.2|AlgiersAlgeria|36.7764|3.0586|
|Africa|Algeria|Algiers|    1|  2|1995|          49.4|AlgiersAlgeria|36.7764|3.0586|
|Africa|Algeria|Algiers|    1|  3|1995|          48.8|AlgiersAlgeria|36.7764|3.0586|
|Africa|Algeria|Algiers|    1|  4|1995|          46.4|AlgiersAlgeria|36.7764|3.0586|
|Africa|Algeria|Algiers|    1|  5|1995|          47.9|AlgiersAlgeria|36.7764|3.0586|
+------+-------+-------+-----+---+----+--------------+--------------+-------+------+
only showing top 5 rows



## Data Cleanup

In [30]:
#We found out that -99 are filler values so any rows with -99 as the Average Temperature should be removed. 
df_final = combined_data.filter(combined_data.AvgTemperature != -99.0)
#df_final.show(5)
df_final.count()

2826655

In [31]:
# Convert date into clean format
from pyspark.sql.functions import concat_ws, to_date, col

#convert "Year" "Month" "Day" to date for use of time serise algorithms 
df_final = df_final.withColumn("date", to_date(concat_ws("-","Year", "Month", "Day")))
# df_final.show(5)
#drop extra colms
df_final = df_final.drop("Month", "Day", "Year", "CityKey")
df_final.show(5)

+------+-------+-------+--------------+-------+------+----------+
|Region|Country|   City|AvgTemperature|    lat|   lng|      date|
+------+-------+-------+--------------+-------+------+----------+
|Africa|Algeria|Algiers|          64.2|36.7764|3.0586|1995-01-01|
|Africa|Algeria|Algiers|          49.4|36.7764|3.0586|1995-01-02|
|Africa|Algeria|Algiers|          48.8|36.7764|3.0586|1995-01-03|
|Africa|Algeria|Algiers|          46.4|36.7764|3.0586|1995-01-04|
|Africa|Algeria|Algiers|          47.9|36.7764|3.0586|1995-01-05|
+------+-------+-------+--------------+-------+------+----------+
only showing top 5 rows



# Save the Result

In [32]:
df_final.coalesce(1).write.format("csv").mode("overwrite").option("header", "true").save("Folder")

# Checking Data is Fully Clean


In [35]:
#Read the new .csv file with the clean data. 
df_final = spark.read.csv('city_temperature_clean.csv', header=True, inferSchema=True)

In [36]:
#If the data was cleaned properly, there should be 2906327 - 79672 = 2826655 rows. 
df_final.count()


2826655

In [37]:
#This cell is to print out the first 5 rows of the new CSV file to double check all the contents were correct put into it. 
df_final.show(5)

+------+-------+-------+--------------+-------+------+-------------------+
|Region|Country|   City|AvgTemperature|    lat|   lng|               date|
+------+-------+-------+--------------+-------+------+-------------------+
|Africa|Algeria|Algiers|          64.2|36.7764|3.0586|1995-01-01 00:00:00|
|Africa|Algeria|Algiers|          49.4|36.7764|3.0586|1995-01-02 00:00:00|
|Africa|Algeria|Algiers|          48.8|36.7764|3.0586|1995-01-03 00:00:00|
|Africa|Algeria|Algiers|          46.4|36.7764|3.0586|1995-01-04 00:00:00|
|Africa|Algeria|Algiers|          47.9|36.7764|3.0586|1995-01-05 00:00:00|
+------+-------+-------+--------------+-------+------+-------------------+
only showing top 5 rows



In [38]:
df_final.printSchema()

root
 |-- Region: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- AvgTemperature: double (nullable = true)
 |-- lat: double (nullable = true)
 |-- lng: double (nullable = true)
 |-- date: timestamp (nullable = true)



# This is where we try to fix the Timestamp to only have the date and no time.


In [52]:
df_final2= df_final.withColumn("date",to_date("date"))

df_final2.show(5)

type(df_final2)

df_final2.printSchema()


+------+-------+-------+--------------+-------+------+----------+
|Region|Country|   City|AvgTemperature|    lat|   lng|      date|
+------+-------+-------+--------------+-------+------+----------+
|Africa|Algeria|Algiers|          64.2|36.7764|3.0586|1995-01-01|
|Africa|Algeria|Algiers|          49.4|36.7764|3.0586|1995-01-02|
|Africa|Algeria|Algiers|          48.8|36.7764|3.0586|1995-01-03|
|Africa|Algeria|Algiers|          46.4|36.7764|3.0586|1995-01-04|
|Africa|Algeria|Algiers|          47.9|36.7764|3.0586|1995-01-05|
+------+-------+-------+--------------+-------+------+----------+
only showing top 5 rows

root
 |-- Region: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- AvgTemperature: double (nullable = true)
 |-- lat: double (nullable = true)
 |-- lng: double (nullable = true)
 |-- date: date (nullable = true)

